这里提供 Tensorflow 基于矩阵乘法实现神经网络的示例代码，输入为列向量

请将其改为输入为行向量的形式

## 运行环境

In [1]:
%load_ext watermark
%watermark

2017-03-23T09:53:57

CPython 2.7.6
IPython 5.3.0

compiler   : GCC 4.8.4
system     : Linux
release    : 4.9.12-moby
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


In [2]:
import tensorflow as tf
print(tf.__version__)

1.0.1


In [3]:
import tensorflow as tf
import numpy as np

In [4]:
def add_nn_layer(input_, weight, bias, activation=None):
    # 假设 input_ 中每一列是一个样本
    raw_output = tf.add(tf.matmul(weight, input_), bias)
    if activation:
        return activation(raw_output)
    else:
        return raw_output

In [7]:
def generate_weight_and_bias(input_dim, output_dim):
    w = tf.Variable(tf.random_uniform([output_dim, input_dim], -1, 1))
    bias = tf.Variable(tf.zeros([output_dim, 1]))
    return (w, bias)

**此处生成一些随机数据，只是为演示所用。最后一个作业的数据请参考 ch3/code/README.md**

这里随机生成两类容易分隔的数据

In [8]:
X = np.concatenate((np.random.rand(2, 50), np.random.rand(2, 50) + 1), axis=1)
# 注意这里要转换成 float32，否则默认可能是精度更高的 float64，和后面 W, b 的类型对应，Tensorflow 的矩阵乘法不会自己做类型转化。
# 也可以在矩阵乘法中使用 tf.cast(x, tf.float32) 来转换类型
X = X.astype(np.float32)
y = np.asarray([0] * 50 + [1] * 50)

## 无隐层神经网络 (Sigmoid + Cross entropy)

In [9]:
W, b = generate_weight_and_bias(2, 1)
nn_output = add_nn_layer(X, W, b, tf.sigmoid)

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    res = sess.run(nn_output)

In [11]:
res.shape

(1, 100)

而之前得到的 (1, 100) 结果不过是把 100 个样本按列进行拼接，一次性得到 100 个样本的结果而已。

### Tensorflow 中的 sigmoid + cross entropy

如果看过作业中 "Neural Network and Deep Learning" Simgoid + cross entropy 相关章节的话，应该能理解，sigmoid + cross entropy 在求导数的时候，可以把分母进行约简，少做一些计算。

Tensorflow 利用了这一特性进行计算约简，因此其有一个专门的 cost 层，叫 `tf.nn.sigmoid_cross_entropy_with_logits`

所谓的 logit，其实是 sigmoid 的反函数，也就是说这个 cost 函数，需要的输入是 sigmoid 变换之后的结果，再取其反函数 logit。

这么这个结果，其实就是我们神经网络输出节点的原始结果，即在 sigmoid 变换之前的结果。


为了配合这个 cost 层进行计算约简，我们需要先准备一个 sigmoid 激活之前的结果，在训练时使用激活之前的结果即可，在『预测』时才使用 sigmoid 激活之后的结果

In [13]:
tf.reset_default_graph()

In [14]:
W, b = generate_weight_and_bias(2, 1)
# X, y 这里是用 numpy array 的方式， Tensorflow 会将他们变成 Constant tensor 放入 graph 当中
# **请尝试将 X, y 改成 tensorflow 的 placeholder 形式**

nn_raw_output = add_nn_layer(X, W, b)
final_output = tf.sigmoid(nn_raw_output)

In [15]:
cost = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=nn_raw_output, labels=y.reshape(1, 100).astype(np.float32))
)

In [16]:
def accuracy(preds, labels):
    return ((preds > 0.5) == labels).sum() / float(len(labels))

In [17]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        sess.run(train_step)
        if i % 50== 0:
            preds = sess.run(final_output)
            print(accuracy(preds, y))

0.5
0.68
0.85
0.94
0.96
0.96
0.98
0.98
0.98
0.99
0.99
0.99
0.99
0.99
0.99
0.99
0.99
0.99
0.99
0.99
